## Manual 버전 데이터셋 제작

해당 코드는 설정한 갯수의 데이터를 기존의 데이터셋(demo/small/large)에서 랜덤하게 가져와 새로운 사이즈(manual - 이름 자유롭게 설정 가능)의 데이터셋을 생성합니다.

## Global Settings

In [1]:
import os
from os import path
import sys

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)
DATA_DIR = path.join(PROJECT_DIR, "data")

from utils import resize_dataset

In [2]:
"""
경로가 폴더를 나타낼 경우 Dir, 파일일 경우 Path로 명명

size: 새로 만들 데이터셋의 크기 (자유롭게 설정 가능)
original_size: 데이터를 가져올 기존 데이터셋의 크기 (demo, small, large 등 이미 있는 것)
"""

size = "onlyone"
original_size = "demo"

datasetDir = path.join(DATA_DIR, "MIND", size)
datasetOriginalDir = path.join(DATA_DIR, "MIND", original_size)

os.makedirs(path.join(datasetDir, "train"), exist_ok=True)
os.makedirs(path.join(datasetDir, "test"), exist_ok=True)

## Prepare parameters

In [15]:
"""
※중요: train_data_number는 2 이상으로 설정해야 합니다.
나중에 데이터 전처리 과정에서 train_test_split()을 통해 train 데이터셋을 train/val로 쪼개기 때문입니다.
"""
train_data_number = 2
test_data_number = 1
random_seed = 5

### 1. manual Train/Test behaviors.tsv 생성

100%|██████████| 1/1 [00:00<?, ?it/s]


### 2. manual Train/Test news.tsv 생성

In [17]:
train_news_df = resize_dataset.generate_news_dataset(
    path.join(datasetDir, "train", "behaviors.tsv"),
    path.join(datasetOriginalDir, "train", "news.tsv"),
    path.join(datasetDir, "train", "news.tsv")
)
test_news_df = resize_dataset.generate_news_dataset(
    path.join(datasetDir, "test", "behaviors.tsv"),
    path.join(datasetOriginalDir, "test", "news.tsv"),
    path.join(datasetDir, "test", "news.tsv")
)

100%|██████████| 1/1 [00:00<?, ?it/s]


### 기타 테스트 코드

In [18]:
print(train_news_df.shape)
print(test_news_df.shape)

(84, 7)
(26, 7)
